In [15]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'http://skodarapidclub.ru/forum/'
folder_names = [
    ['0', 'Отзывы', 'f=28', 'SKODA RAPID'],
    ['1', 'Двигатель', 'f=29', 'SKODA RAPID'],
    ['2', 'Трансмиссия', 'f=30', 'SKODA RAPID'],
    ['3', 'Ходовая', '', 'SKODA RAPID'],
    ['4', 'Колеса', 'f=34', 'SKODA RAPID'],
    ['5', 'Электрооборудование', 'f=33', 'SKODA RAPID'],
    ['6', 'Мультимедиа', '', 'SKODA RAPID'],
    ['7', 'Кузов', 'f=31', 'SKODA RAPID'],
    ['8', 'Жидкости', '', 'SKODA RAPID'],
    ['9', 'Сервис', 'f=24', 'SKODA RAPID'],
    ['10', 'Эксплуатация', 'f=23', 'SKODA RAPID'],
    ['11', 'Тюнинг', 'f=24', 'SKODA RAPID'],
    ['12', 'Кузов-Салон', 'f=32', 'SKODA RAPID'],
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [16]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [17]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем некоторые метаданные сообщений на текущей странице
            tables = soup.find_all('table')
            for next_table in tables:
                if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                    post_column = next_table.find('td')
                    post_date = post_column.get_text().strip()
                    post_data[re.sub('post', '', next_table['id'])] = [post_date]
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        post_date = post_data[message_id][0]
                        author = post_data[message_id][1][1]
                    else:
                        post_date = ''
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [18]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [19]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [20]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

The ipytelegram extension is already loaded. To reload it, use:
  %reload_ext ipytelegram


In [21]:
%%telegram_send Next message
scraper(site, folder_names)

Дрожь руля.
Работа охранником
Мотор потряхивает на холостых. Мигает чек.
Девушки
Элитные малышки
Банька и девки
Люблю стройных
Звуковая паранойя. Дилер ничего не слышит.
Что-то потрескивает в колёсах.
Передние тормозные колодки
Гул (подвывание) снизу после продолжительных поездок
не заводится рапид
Лобовое стекло
Каталожные номера расходников и запчастей
Хэлп, мои новые друзья))
При повороте замка зажигания не заводится двигатель
Обогрев в салоне
Двигатель не заводится
омыватель лобового стекла
Увиличение расхода топлива
Педаль газа.
контрольные лампы Шкоды Рапид
Вибрация в салоне
Загорелись ABS/ESC; подушка безопасности; OPS
Не горят ДХО
Двери и аккумулятор
Шумит вентилятор кондиционера.
Антифриз
Вопрос.
Расширительный бачок
Свечи , калийное число
Кондей
Датчик окружающей температуры
Установка фаркопа
Кодировка блока BCM номер 6C0937089C
Не смывается полоса от крайнего положения маленького дворника(щетки стеклоочистителя)
Неисправность - постронний звук
Акум/стартер/генератор?
Есть ли

Не заводится автомобиль
Детонация двигателя
течь антифриз
Ошибка тормозной системы в ремонт
Указатель уровня топлива глючит
Вентилятор на максимум после вебасто
Нужна консультация Указатель уровня топлива глючит
нужна консультация.
Нужна консультация
Моторное масло для Шкода Рапид
Обязательная замена топливного фильтра
Информация о двигателях
The Importance of Writing Well
spm english essay story
100 good research paper topics history us
help write my essay for me cheap discount code
free essays 1984
pay to do cheap essay on shakespeare
thesis statement for legalizing marijuana for medical usage
stress and health essay
anne boleyn thesis topics
thesis proposal writing for hire usa
hundred years war essay topics http //www.history.com/topics/hundred-years-war
how to write a short essay for college application us
Human Resource (HR) Build an Ethical Culture
write an expository essay on my favorite food friend
need help writing a thesis statement
write me astronomy dissertation
thesis fon

Перепрошивка с 90 л.с на 110
Работа двигателя 1.4 на холостом ходу
Цоканье на холодную
Подделки моторных масел
Двигатель кушает масло
Писк из под капота + щелчки при включении и выключении зажигания
Защита двигателя Шкода Рапид
Замена двигателя 1.2 TSI на 1.4 FSI
Расход масла на Шкода Рапид
стук CFNA 1.6
Датчик ДПКВ ДТОЖ
Установка газобалонного оборудования (ГБО) на 1,6 CVWA 110 л.с.
Включение вентилятора
обороты для переключения передачи
загорелась лампа падения давления в системе смазки
Help. Глючит антибукс, или датчик пробуксовки,  ТС
Датчик
Каталожные номера для первого ТО
Топливный бак
Маслянный фильтр
холодный пуск и гарантия
Появился какой то дополнительный шум в двигателе!
Гул или шум от двигателя
98 -ой бензин Skoda Rapid 1.4
ГБО актуально? Изменения?
СРОЧНО. НОВЫЙ ДВИГАТЕЛЬ CFW PL 1.6 - 90 л.с. Надежен ли ? И чем отличие от CFNA?
Замена масла  в двигателе
чудо чудное
Сброс сообщения о замене масла
Машина не заводится!!!
CFNA механика. стук в двигателе на холодную, выжимной р

Свист тормозов
Проконсультируйте по маслу Febi
Rapid 2016г 1.6/110лс Ошибка коробки передач
Прерывистый свист при торможении
Проблемы с автоматом 1.6 mpi
Трест DSG DQ200 при переключении
скрип на кочках
Скрип, гул при повороте руля
Машина катится при стоянке на скорости МКПП
Как правильно обслуживать АКПП AQ160-6F 09G – муки выбора!
Хруст и стук в районе колеса
Лопнул левый привод
Хруст в рулевой
QDM АКПП AQ160-6F 09G на Рапидах после 2017 г. выпуска
АКПП, писк, масло
Смещение центра руля
ошибка электроусилителя руля
Вибрация кузова
пробой передней подвески
Акпп упала в аварийный режим
Умерли амортизаторы
Опыт замены жидкости в АКПП
АКПП клюёт машина
Толчки при переключении передач на автомате
АКПП помогите найти номер детали, или где купить?
Переключение передач на высоких оборотах (АКПП)
Акпп упала в аварийный режим
Подбор тормозных колодок
Скрип при трогании с места
Вопрос по работе коробки автомат
сломался теплообменник, акпп умерла
Шум МКПП
Руль
Ошибка усилителя руля
Замена сцепле

Комплект на зиму.
Rapid 2020 и диски 6x15/5x100 ET 40
Оригинальные диски Skoda Rapid
Прошу совета в выборе зимних шин R 16
Литые диски на рапид
Размеры шин, резины для Шкода Рапид
essay paper writing service
write my paper essayerudite.com
essay writing service essayerudite.com
essay writing service. essay help
write my paper. paper writing service
essay writing service 2020
write my paper 2020
театр ростовый кукол
РЕКЛАМА В ЯНДЕКС ДИРЕКТ И GOOGLE ADS - ЦЕНЫ В 2-3 РАЗА НИЖЕ РЫНОЧНЫХ
На какой летней резине вы ездите?
Продам комплект колес
Продаю летние колеса
Продам летние шины Continental ContiPremiumContact 195/55 R15
Нужен совет по дискам r17.
Литье на рапид
Какую резину можно поставить на оригинальные диски r16
Ищу резину Bridgestone b250sz 195/55/r15 - 1-2шт (Спб)
Ключ для секретки
Продам колеса от rapida R15 Недорого!
Поменять штампы на литье, подскажите не которые моменты
Продам летние шины Bridgestone turanza T001 195/55 R16
Посоветуйте где купить диски
Цепи противоскольжения на

...

http://velosiped.cf/
Public pretty woman you hurt me scene
Что лучше качественнаяя копия или оригинал?
http://kurtki.ml/
Полировка
http://lifchik.tk/
http://product-of-kitay.cf/
http://rabotay.cf/
муром ритуальные услуги
рославль ритуальные услуги
южноуральск ритуальные услуги
https://clck.ru/R69em Ритуальный агент
http://whatsapplending.gq/
http://whatsapplanding.gq/
Нужна помощь!
вода в салоне.
Сетка в бампер и решетку радиатора
Декоративная накладка буксировочного крюка
Помогите с краской от сколов
Скол на лобовом стекле.
Ремонт после ДТП.
что то гремит
отверстие в багажнике
Задний дворник
Дворники VAG 5JB 998 001
Газовые упоры капота.
Защита картера (двигателя) для Рапид!
вот такая экономия "Замена бокового зеркала заднего вида"
Крышка багажника
Дверь поцарапала порог
Как называется  нижняя часть подкрылка?
Кузовные детали от старого рапида к рестайлингу
Подходят ли дефлекторы со старого кузова на новый 2018 года
Заглушка
Погнул порог домкратом
Царапины
Разные зазоры крышки багажни

Атлант-М Тушино: официальная ветка и скидки
Клиентская поддержка АвтоСпецЦентр Марьино
Проверка дилеров
Дилеры Шкода в Москве
Где предпочтительнее обслуживать автомобиль?
ВЕНТУС - официальный дилер SKODA
Дилеры Шкода в Самаре
Ремонт после ДТП
АвтоСпецЦентр Каширка - скидки клубу 15% на работы и запчасти
Отрицательный опыт покупки Рапида.
Общение с официальным представителем Автопрага Skoda
ТО 2 к кому ехать?
Обслуживание в АвтоСпецЦентр Внуково, Официальный дилер ŠKODA
Автоцентр "Злата" Нижний Новгород /ТО/обслуживание/акции
Выбор дилера для ТО
Автоспеццентр Химки!
Дилеры Шкода в Санкт-Петербурге
Электронная сервисная книжка и отметки в ней.
Дилеры компетентнее производителя?
Сигма Сервис, г. Санкт-Петербург
АвтоСертификат "ЗАЩИТА" от АвтоСпецЦентр - развод !!
фирменная присадка в топливо
Дилер "Мотом" Ульяновск
Неон-Авто | Пулково-Авто | Таллинский-Авто (ГК Wagner, Санкт-Петербург)
10% Скидка всем форумчанам!!!
Дилеры Шкода в Пензе
Тайный Покупатель!!!
Дилеры Шкода в Саратове
Предложе

Увеличился расход топлива
Задний стеклоочеститель. За и Против??
Работа кондиционера в режиме охлаждения
Руководства по ремонту Skoda Rapid
Эпопея с продажей.
Кто как обкатывал новый двигатель?
Не работает подогрев форсунок омывателя?!
Рапид зимой! Делимся впечатлениями
Rapid 1.6 110 л.с. Hockey Edition с новым двигателем 2015г.
Проблема с заменой лампы ближнего света
Чем отмыть авто...
расход масла 1.6 110лошадей
Skoda Rapid 1.4 TSI DSG-7
Не работает омыватель лобового и заднего стекла
Прицеп для Рапида
Покупка авто
Дворник на задней двери (стекле)
Шум МКПП
Чехлы Санкт-Петербург
Охранный комплекс
Запуск двигателя, писк, Rapid 2016
Щелчки под машиной...
Ассистент торгания на подъеме
отопитель и двигатель
Плюсы и минусы, проблемы и недостатки Шкода Рапид
Рапид 2018г
Непруха...
Нарушение в работе вентилятора.
WIN номер
Нулевое ТО, 7000км. Видеоотчёт
Пассажирские перевозки Крым Киев
Отчёт по пробегу в 5000 км
АВТОБАФФЕРЫ
Сел аккумулятор и работает сигнализация
Skies Rapid 1,2 active
Ощуще

Атлант-М Тушино: официальная ветка и скидки
Клиентская поддержка АвтоСпецЦентр Марьино
Проверка дилеров
Дилеры Шкода в Москве
Где предпочтительнее обслуживать автомобиль?
ВЕНТУС - официальный дилер SKODA
Дилеры Шкода в Самаре
Ремонт после ДТП
АвтоСпецЦентр Каширка - скидки клубу 15% на работы и запчасти
Отрицательный опыт покупки Рапида.
Общение с официальным представителем Автопрага Skoda
ТО 2 к кому ехать?
Обслуживание в АвтоСпецЦентр Внуково, Официальный дилер ŠKODA
Автоцентр "Злата" Нижний Новгород /ТО/обслуживание/акции
Выбор дилера для ТО
Автоспеццентр Химки!
Дилеры Шкода в Санкт-Петербурге
Электронная сервисная книжка и отметки в ней.
Дилеры компетентнее производителя?
Сигма Сервис, г. Санкт-Петербург
АвтоСертификат "ЗАЩИТА" от АвтоСпецЦентр - развод !!
фирменная присадка в топливо
Дилер "Мотом" Ульяновск
Неон-Авто | Пулково-Авто | Таллинский-Авто (ГК Wagner, Санкт-Петербург)
10% Скидка всем форумчанам!!!
Дилеры Шкода в Пензе
Тайный Покупатель!!!
Дилеры Шкода в Саратове
Предложе

Не греет печка
Спортивные сидения Dynamic
перестала дуть печка
объяните за climatronic
Выбор недорогих ковриков
Подлокотник 5JA061123A
Пенал - "Бардачок" сверху в торпедо.
Водительская дверь
Дребезги, сверчки, скрип в салоне
Кондишн/печка
Тюнинг салона
Автомобильные чехлы.
Мне не комфортно на водительском сиденьи
Очень не приятный звук в салоне
Чехлы на спортивные сидения
Свист потоков воздуха внутри салона при высоких оборотах вентилятора климата/печки
Во время работы печки сам включается кондиционер
Приборная панель
Как защитить спортивные сидения от ног детей на 2 ряду?
Накладки на пороги
Поломка на панели
Чехлы на Стайл.
Шумоизоляция Skoda Rapid
Шум при скорости
Скрип руля
Раздельная спинка 60/40
Демонтировать диван
Багажник Rfpid
Салон
Тент для Rapid
Тюнинг деталей салона
Пластик
подлокотник
Артикул заглушки
Сиденья
Хром кольца в приборку
Оригинальные коврики на шкоду рапид
Второй плафон освещения багажника
Что за деталь такая?
Проблема неудобных передних подголовников
Как снять о

In [ ]:
import pandas as pd
mess = pd.read_csv('./LADA VESTA/LADA VESTA_10_Эксплуатация.csv', quotechar="'")
mess

In [6]:
folder_names = [['12', 'Кузов-Салон', 'f=32', 'SKODA RAPID']]
folder = folder_names[0][2]
current_themes = collect_themes(site, folder)
current_themes

[['Не греет печка', 'http://skodarapidclub.ru/forum/showthread.php?t=126196'],
 ['Спортивные сидения Dynamic',
  'http://skodarapidclub.ru/forum/showthread.php?t=122950'],
 ['перестала дуть печка',
  'http://skodarapidclub.ru/forum/showthread.php?t=125911'],
 ['объяните за climatronic',
  'http://skodarapidclub.ru/forum/showthread.php?t=126084'],
 ['Выбор недорогих ковриков',
  'http://skodarapidclub.ru/forum/showthread.php?t=1626'],
 ['Подлокотник 5JA061123A',
  'http://skodarapidclub.ru/forum/showthread.php?t=1254'],
 ['Пенал - "Бардачок" сверху в торпедо.',
  'http://skodarapidclub.ru/forum/showthread.php?t=1688'],
 ['Водительская дверь',
  'http://skodarapidclub.ru/forum/showthread.php?t=1206'],
 ['Дребезги, сверчки, скрип в салоне',
  'http://skodarapidclub.ru/forum/showthread.php?t=426'],
 ['Кондишн/печка', 'http://skodarapidclub.ru/forum/showthread.php?t=122651'],
 ['Тюнинг салона', 'http://skodarapidclub.ru/forum/showthread.php?t=39'],
 ['Автомобильные чехлы.',
  'http://skodar

In [ ]:
themes = []
flag = True

while flag:
    
    resp = urlopen(site_address) # скачиваем файл
    html = resp.read().decode('utf-8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
    flag = False
    for link in soup.find_all('a'):
        if link.has_attr('href') and link.has_attr('id'):
            if re.search('thread_title_', str(link.get('id'))):
                s = link.get('href')
                s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                themes.append([link.get_text(), s])
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]

In [9]:
themes = [['Выбор недорогих ковриков',
  'http://skodarapidclub.ru/forum/showthread.php?t=1626']]
current_messages = collect_messages(site, current_themes)
current_messages

Не греет печка
Спортивные сидения Dynamic
перестала дуть печка
объяните за climatronic
Выбор недорогих ковриков
Подлокотник 5JA061123A
Пенал - "Бардачок" сверху в торпедо.
Водительская дверь
Дребезги, сверчки, скрип в салоне
Кондишн/печка
Тюнинг салона
Автомобильные чехлы.
Мне не комфортно на водительском сиденьи
Очень не приятный звук в салоне
Чехлы на спортивные сидения
Свист потоков воздуха внутри салона при высоких оборотах вентилятора климата/печки
Во время работы печки сам включается кондиционер
Приборная панель
Как защитить спортивные сидения от ног детей на 2 ряду?
Накладки на пороги
Поломка на панели
Чехлы на Стайл.
Шумоизоляция Skoda Rapid
Шум при скорости
Скрип руля
Раздельная спинка 60/40
Демонтировать диван
Багажник Rfpid
Салон
Тент для Rapid
Тюнинг деталей салона
Пластик
подлокотник
Артикул заглушки
Сиденья
Хром кольца в приборку
Оригинальные коврики на шкоду рапид
Второй плафон освещения багажника
Что за деталь такая?
Проблема неудобных передних подголовников
Как снять о

[["'Не греет печка'",
  [['02.12.2020, 22:27',
    'Елизавета',
    "'Добрый день! Возможно кто-то сталкивался с данной проблемой, в автосервисах помочь не могут ( В радиатор печки был добавлен герметик, после чего печка перестала подавать теплый воздух в салон. Радиатор был заменен, но не на оригинал. Проблема не решилась, печка как не думал теплым, так и не дует. В сервисе попробовали сделать промывку, но ничего не поменялось. Может ли быть проблема в теплоотдаче не оригинального радиатора ?'"]]],
 ["'Спортивные сидения Dynamic'",
  [['11.02.2020, 12:12',
    'um0krh',
    "'Всем привет. Хотел поинтересоваться никто не задавался вопросов покупки данных сидений отдельно? Именно не доработки стандартных а покупкой новых. Позвонил официальному дилеру в москве, мне сказали что если их покупать отдельно, а не заказывать при покупке автомобиля то их стоимость составит порядка 180 000 рублей Вот мне интересно человек который меня консультировал был пьяный или шкода действительно не в своем 

In [ ]:
messages = []
for theme in tqdm(themes):
    theme_address = theme[1]
    thread = re.search('t=[0-9]+', theme_address)[0]
    flag = True
    theme_messages = []
    post_data = {}
    while flag:
        resp = urlopen(theme_address) # скачиваем файл
        html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
        #Собираем некоторые метаданные сообщений на текущей странице
        tables = soup.find_all('table')
        for next_table in tables:
            if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                post_column = next_table.find('td')
                post_date = post_column.get_text().strip()
                post_data[re.sub('post', '', next_table['id'])] = [post_date]
        #Собираем сами сообщения
        divs = soup.find_all('div')
        for div in divs:
            if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                continue
            if div.has_attr('id') and re.search('postmenu_', div['id']):
                author = div.find('a')
                if author != None:
                    author_id = re.search('u=[0-9]+', author['href'])[0]
                    author_nickname = author.get_text()
                else:
                    author_id = div.get_text().strip()
                    author_nickname = 'Guest'
                post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                print(re.sub('postmenu_', '', div['id']), post_data[re.sub('postmenu_', '', div['id'])])
            if div.has_attr('id') and re.search('post_message_', div['id']):
                message_id = re.sub('post_message_', '', div['id'])
                if message_id in post_data:
                    post_date = post_data[message_id][0]
                    author = post_data[message_id][1][1]
                else:
                    post_date = ''
                    author = ''
                for item in div.children:
                    if item.name == 'div':
                        item.clear()
                s = re.sub('[\']+', '`', div.get_text())
                s = '\'' + re.sub('[\s]+', ' ', s) + '\''
                if len(s) > 0:
                    theme_messages.append([post_date, author, s])
        # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
        # Если ссылка найдена, 
        flag = False 
        for link in soup.find_all('a'):
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
    messages.append(['\''+ theme[0] + '\'', theme_messages])

In [ ]:
messages

In [85]:
f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
with open(f_name, 'w', encoding='utf-8') as ouf:
    ouf.write('Code,Folder,Theme,Date,Author,Message\n')
    for message in messages:
        header = folder_name[:2] + [message[0]]
        for item in message[1]:
            ouf.write(','.join(header + item))
            ouf.write('\n')

In [69]:
folder_name = ['0', 'Отзывы', 'f=25', 'LADA VESTA']
f_name = folder_name[3] + '/' + folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

'LADA VESTA/LADA VESTA_0_Отзывы.csv'

In [ ]:
'''messages = []
theme_address = 'http://www.lada-vesta.net/showthread.php?t=9571'
resp = urlopen(theme_address) # скачиваем файл
html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп
divs = soup.find_all('div')
for div in divs:
    if div.has_attr('id'):
        if re.search('post_message_', div['id']):
            for item in div.children:
                if item.name != 'div':
                    s = str(item).strip()
                    if len(s) > 0:
                        messages.append([s])
print(messages)'''